In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
import plotly.io
plotly.io.renderers.default = 'notebook_connected'

In [3]:
datestr='4-april-2020'



In [4]:
today = pd.to_datetime(datestr.replace('-', ' '))

In [5]:
CASE_URL_FORMAT = 'https://www.health.govt.nz/system/files/documents/pages/covid-19-case-details-update-%s.xlsx'

In [6]:
with pd.ExcelFile(CASE_URL_FORMAT % datestr) as excel_file:
    cases_dfs = {name: excel_file.parse(i, header=3) for i, name in enumerate(['confirmed', 'probable'])}

In [7]:
def get_case_counts(df, date_name='Report Date', date_col=0):
    return df.groupby([df.columns[date_col], 'DHB']).agg(c=('DHB', 'count')).c.rename_axis([date_name, 'DHB'])

In [8]:
cdf = pd.concat({key: get_case_counts(df) for key, df in cases_dfs.items()}, axis=1, names=['type']).loc[:today - pd.Timedelta(days=1)]

In [9]:
totals_df = cdf.sum(1).rename('cases').reset_index()

In [10]:
px.bar(totals_df, x='Report Date', y='cases', color='DHB')

In [11]:
px.bar(totals_df, color='Report Date', y='cases', x='DHB', color_continuous_scale = px.colors.sequential.algae)

In [12]:
tdf = cdf.unstack('DHB').fillna(0).cumsum().stack('DHB')





In [13]:
tdf = tdf.assign(total = tdf.sum(1)).reset_index()
tdf = tdf[tdf.total>0]


In [14]:
tdf

type,Report Date,DHB,confirmed,probable,total
0,2020-02-26,Auckland,1.0,0.0,1.0
20,2020-02-28,Auckland,1.0,0.0,1.0
32,2020-02-28,Southern,2.0,0.0,2.0
40,2020-03-02,Auckland,1.0,0.0,1.0
52,2020-03-02,Southern,2.0,0.0,2.0
...,...,...,...,...,...
575,2020-04-03,Waikato,126.0,9.0,135.0
576,2020-04-03,Wairarapa,6.0,2.0,8.0
577,2020-04-03,Waitemata,100.0,29.0,129.0
578,2020-04-03,West Coast,4.0,0.0,4.0


In [15]:
xdf = cdf.unstack('DHB').fillna(0).cumsum()
xdf = xdf.stack(['DHB', 'type'])
xdf = xdf[xdf>0].unstack('type')
sdf = xdf.sum(0, level='Report Date').assign(DHB='total')
xdf = xdf.append(sdf.set_index('DHB', append=True), sort=True)
xdf = xdf.assign(total=xdf.sum(1)).reset_index().fillna(0)

In [16]:
px.line(xdf, x='Report Date', y='total', color='DHB', hover_data=['confirmed', 'probable'], log_y=True)

In [17]:
fig=px.bar(xdf[xdf.DHB!='total'], x='Report Date', y='total', color='DHB')
fig.data[0]

Bar({
    'alignmentgroup': 'True',
    'hoverlabel': {'namelength': 0},
    'hovertemplate': 'DHB=Auckland<br>Report Date=%{x}<br>total=%{y}',
    'legendgroup': 'Auckland',
    'marker': {'color': '#636efa'},
    'name': 'Auckland',
    'offsetgroup': 'Auckland',
    'orientation': 'v',
    'showlegend': True,
    'textposition': 'auto',
    'x': array([datetime.datetime(2020, 2, 26, 0, 0),
                datetime.datetime(2020, 2, 28, 0, 0),
                datetime.datetime(2020, 3, 2, 0, 0),
                datetime.datetime(2020, 3, 4, 0, 0),
                datetime.datetime(2020, 3, 5, 0, 0),
                datetime.datetime(2020, 3, 6, 0, 0),
                datetime.datetime(2020, 3, 12, 0, 0),
                datetime.datetime(2020, 3, 13, 0, 0),
                datetime.datetime(2020, 3, 14, 0, 0),
                datetime.datetime(2020, 3, 15, 0, 0),
                datetime.datetime(2020, 3, 16, 0, 0),
                datetime.datetime(2020, 3, 17, 0, 0),
              

In [18]:
DHBS = ['']

In [19]:
df.groupby('DHB').count()

NameError: name 'df' is not defined

In [ ]:
cdf.groupby(['Report Date', 'DHB']).agg(count=('DHB', 'count')).cumsum()

In [ ]:
px.bar(cdf.groupby(['Report Date', 'DHB']).agg(count=('DHB', 'count')).reset_index(), x='Report Date', y='count', color='DHB')

In [ ]:
df2.groupby('Report Date').agg(count=('DHB', 'count'))

In [ ]:
df2.groupby('DHB').count()